In [1]:
import Setup
import pandas as pd
import numpy as np
from datetime import datetime,date
import matplotlib.pyplot as plt
import glob
import re

/Users/calumthompson/Google Drive/Fantasy_football/Setup.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# Update github data

In [2]:
! git -C {Setup.github_address} pull

print('last updated: ')
! git -C {Setup.github_address} show --no-patch --no-notes --pretty='%cd'


remote: Enumerating objects: 1241, done.
remote: Counting objects: 100% (1241/1241), done.
remote: Compressing objects: 100% (532/532), done.
remote: Total 1241 (delta 613), reused 1241 (delta 613), pack-reused 0
Receiving objects: 100% (1241/1241), 278.62 KiB | 1.14 MiB/s, done.
Resolving deltas: 100% (613/613), completed with 586 local objects.
From https://github.com/vaastav/Fantasy-Premier-League
   d2ac2856f..40ecf5081  master     -> origin/master
Updating d2ac2856f..40ecf5081
Fast-forward
 data/2020-21/cleaned_players.csv                   |  736 ++++++------
 data/2020-21/fixtures.csv                          |  126 +-
 data/2020-21/gws/gw10.csv                          |  604 ++++++++++
 data/2020-21/gws/merged_gw.csv                     |  603 ++++++++++
 data/2020-21/gws/xP10.csv                          |  604 ++++++++++
 data/2020-21/player_idlist.csv                     |    2 +
 data/2020-21/players/Aaron_Connolly_78/gw.csv      |    1 +
 data/2020-21/players/Aaron_Cressw

# Fixtures and teams

## Transform fixture table

In [3]:
# Open table

Date_format = lambda x: pd.datetime.strptime(x, "%d/%m/%Y %H:%M")

data_merge = []

for i in Setup.fixtures_link:
    
    print('Importing ',i)
    
    link = Setup.fixtures_link[i]
    
    import_df = pd.read_csv(link,parse_dates = ['Date'], date_parser = Date_format)
    
    import_df['FIX_season'] = i
    
    data_merge.append(import_df)
    
# Split scores into home and away columns 
fixtures_RAW = pd.concat(data_merge)
fixtures_RAW[['home_score' , 'away_score']] = fixtures_RAW['Result'].str.split(' - ', expand = True)

fixtures_RAW['home_score'] = pd.to_numeric(fixtures_RAW['home_score'])
fixtures_RAW['away_score'] = pd.to_numeric(fixtures_RAW['away_score'])

# Transform shape of table to order by team + date
merge = []

# Split into individual seasons
for iter_season in fixtures_RAW['FIX_season'].unique():
    
    season_df = fixtures_RAW.loc[fixtures_RAW['FIX_season'] == iter_season]
    

    for iter_team in season_df['Home Team'].unique():
        
        team_record = season_df.loc[(season_df['Home Team'] == iter_team) | (season_df['Away Team'] == iter_team)]

        col_team = np.full(38, iter_team)
        col_GWs = np.arange(1,39)
        col_date = team_record['Date'].dt.date
        col_Opponent = np.where(team_record['Home Team'] == iter_team, team_record['Away Team'],team_record['Home Team'])
        col_home = np.where((team_record['Home Team'] == iter_team), 1 , 0)
        col_goals_marg = np.where((team_record['Home Team'] == iter_team), team_record['home_score'], team_record['away_score'])
        col_goals_cum  = np.cumsum(col_goals_marg)
        col_opp_goals = np.where((team_record['Home Team'] == iter_team), team_record['away_score'], team_record['home_score'])
        col_points_marg = np.where(col_goals_marg > col_opp_goals, 3, 
                                     np.where(col_goals_marg == col_opp_goals, 1, col_goals_marg))
        col_points_cum = np.cumsum(col_points_marg)

        
        team_record = pd.DataFrame({'FIX_season' : iter_season
                                   ,'FIX_team':col_team
                                   ,'FIX_GW':col_GWs
                                   ,'FIX_Fixture_date':col_date
                                   ,'FIX_Opponent':col_Opponent
                                   ,'FIX_Home?':col_home
                                   ,'FIX_goals_marginal' : col_goals_marg
                                   ,'FIX_goals_cumulative':col_goals_cum
                                   ,'FIX_opp_goals' : col_opp_goals
                                   ,'FIX_points_marginal' : col_points_marg
                                   ,'FIX_points_cumulative' : col_points_cum })

        merge.append(team_record)
    
fixtures_df = pd.concat(merge).reset_index(drop = True)    

Importing  2020-21


/Users/calumthompson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Importing  2019-20


/Users/calumthompson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in greater


## Compute cumulative points and goals

In [4]:
# Copy and merge to get cumulative points and goals for opponents

opponent_points_cumulative = fixtures_df[['FIX_team','FIX_Fixture_date','FIX_goals_cumulative','FIX_points_cumulative']].copy()


opponent_points_cumulative = opponent_points_cumulative.rename(columns = {'FIX_team' : 'FIX_Opponent',
                                                                          'FIX_goals_cumulative' : 'FIX_opponent_goals_cumulative',
                                                                          'FIX_points_cumulative' : 'FIX_opponent_points_cumulative'})
fixtures_df = fixtures_df.merge(opponent_points_cumulative)

In [5]:
# Shift by one week for incoming points total 
fixtures_df['INCOMING_team_points'] = fixtures_df['FIX_points_cumulative'].shift(1)
fixtures_df['INCOMING_opponent_points'] = fixtures_df['FIX_opponent_points_cumulative'].shift(1)

# Shift by one week for incoming goal totals
fixtures_df['INCOMING_team_goals'] = fixtures_df['FIX_goals_cumulative'].shift(1)
fixtures_df['INCOMING_opponent_goals'] = fixtures_df['FIX_opponent_goals_cumulative'].shift(1)

# Calculate ratio between team/opponent goals/points
fixtures_df['INCOMING_points_diff'] = fixtures_df['INCOMING_team_points']-fixtures_df['INCOMING_opponent_points']
fixtures_df['INCOMING_goals_diff'] = fixtures_df['INCOMING_team_goals']-fixtures_df['INCOMING_opponent_goals']

## Map teams to Vaastav IDs

In [6]:
team_ids = Setup.team_ids.copy()

opponent_ids = Setup.team_ids.copy()
opponent_ids = opponent_ids.rename(columns = {'FIX_team' : 'FIX_opponent_team',
                                              'team_ID' : 'FIX_opponent_code'}) 

In [7]:
fixtures_df = pd.merge(fixtures_df, team_ids, how = 'left',
         left_on = ['FIX_season','FIX_team'] , right_on = ['FIX_season', 'FIX_team'])

fixtures_df = pd.merge(fixtures_df, opponent_ids, how = 'left',
         left_on = ['FIX_season','FIX_Opponent'] , right_on = ['FIX_season', 'FIX_opponent_team'])

## Downfill team points


In [8]:
fixtures_df['INCOMING_team_points'] = fixtures_df['INCOMING_team_points'].ffill()
fixtures_df['INCOMING_opponent_points'] = fixtures_df['INCOMING_opponent_points'].ffill()
fixtures_df['INCOMING_team_goals'] = fixtures_df['INCOMING_team_goals'].ffill()
fixtures_df['INCOMING_opponent_goals'] = fixtures_df['INCOMING_opponent_goals'].ffill()
fixtures_df['INCOMING_points_diff'] = fixtures_df['INCOMING_points_diff'].ffill()
fixtures_df['INCOMING_goals_diff'] = fixtures_df['INCOMING_goals_diff'].ffill()

## Fixture difficulty

In [9]:
input_array = []

for i in Setup.Fixture_difficulties:
    load_address = Setup.Fixture_difficulties[i]
    
    load = pd.read_csv(load_address, parse_dates = ['kickoff_time'], usecols= ['kickoff_time','team_a','team_a_difficulty','team_h','team_h_difficulty'])
    
    load['kickoff_time'] = load['kickoff_time'].dt.date
    
    load['FIX_season'] = i
    
    input_array.append(load)

fix_difficulties = pd.concat(input_array)

transform_merge = []

for team_id in team_ids['team_ID']:
    
    team_diff = fix_difficulties.loc[(fix_difficulties['team_a'] == team_id) | (fix_difficulties['team_h'] == team_id)]
    opponent = np.where(team_diff['team_a'] == team_id, team_diff['team_h'], team_diff['team_a'])
    home = np.where(team_diff['team_a'] == team_id, 0, 1)
    difficulty = np.where(team_diff['team_a'] == team_id, team_diff['team_a_difficulty'], team_diff['team_h_difficulty'])
    
    transform_merge.append(pd.DataFrame({'FIX_season' : team_diff['FIX_season'],
                                         'team_ID' : team_id,
                                         'opponent_ID' : opponent,
                                         'home' : home,
                                         'difficulty' : difficulty}))    

fix_difficulties = pd.concat(transform_merge).drop_duplicates()

fixtures_df = pd.merge( fixtures_df, fix_difficulties,
             left_on = ['team_ID' , 'FIX_opponent_code','FIX_season','FIX_Home?'],
             right_on = ['team_ID','opponent_ID','FIX_season','home'])

## Tidy

In [10]:
fixtures_df = fixtures_df[['FIX_season',                        
                           'FIX_GW', 
                           'FIX_Fixture_date', 
                           'team_ID',
                           'FIX_team', 
                           'FIX_opponent_code',
                           'FIX_Opponent', 
                           'FIX_Home?',
                           'difficulty',
#                            'FIX_goals_marginal', 
#                            'FIX_goals_cumulative', 
#                            'FIX_opp_goals', 
#                            'FIX_points_marginal',
#                            'FIX_points_cumulative', 
#                            'FIX_opponent_goals_cumulative', 
#                            'FIX_opponent_points_cumulative',
                           'INCOMING_team_points', 
                           'INCOMING_opponent_points', 
                           
                           'INCOMING_team_goals',
                           'INCOMING_opponent_goals', 
                           
                           'INCOMING_points_diff', 
                           'INCOMING_goals_diff']]

fixtures_df.head()

,FIX_season,FIX_GW,FIX_Fixture_date,team_ID,FIX_team,FIX_opponent_code,FIX_Opponent,FIX_Home?,difficulty,INCOMING_team_points,INCOMING_opponent_points,INCOMING_team_goals,INCOMING_opponent_goals,INCOMING_points_diff,INCOMING_goals_diff
0,2020-21,1,2020-09-12,8,Fulham,1,Arsenal,1,3,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-21,2,2020-09-19,8,Fulham,10,Leeds,0,3,0.0,3.0,0.0,3.0,-3.0,-3.0
2,2020-21,3,2020-09-28,8,Fulham,2,Aston Villa,1,3,3.0,6.0,3.0,7.0,-3.0,-4.0
3,2020-21,4,2020-10-04,8,Fulham,20,Wolves,0,3,3.0,6.0,3.0,7.0,-3.0,-4.0
4,2020-21,5,2020-10-18,8,Fulham,15,Sheffield Utd,0,2,3.0,7.0,3.0,4.0,-4.0,-1.0


# Get player data


## Player details

In [11]:
data_merge = []

for iter_season in Setup.players_raw_addresses:
    
    PR_path = Setup.players_raw_addresses[iter_season]
    print(PR_path)

    import_df = pd.read_csv(PR_path,encoding = "ISO-8859-1", usecols = ['id',
                                                                        'element_type',
                                                                        'first_name',
                                                                        'second_name', 
                                                                        'form', 
                                                                        'team',
                                                                        'now_cost',
                                                                        'in_dreamteam'])
    import_df['FIX_season'] = iter_season

    data_merge.append(import_df)
    
player_details = pd.concat(data_merge)
player_details = player_details.rename(columns = {'element_type' : 'position',
                                                'id': 'player_ID',
                                                'team' : 'team_ID',
                                                'now_cost' : 'cost_today'})

player_details.head()

/Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2019-20/players_raw.csv
/Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2020-21/players_raw.csv


,position,first_name,form,player_ID,in_dreamteam,cost_today,second_name,team_ID,FIX_season
0,2,Shkodran,1.4,1,False,51,Mustafi,1,2019-20
1,2,HÃ©ctor,1.6,2,False,52,BellerÃ­n,1,2019-20
2,2,Sead,2.6,3,False,52,Kolasinac,1,2019-20
3,2,Ainsley,0.4,4,False,45,Maitland-Niles,1,2019-20
4,2,Sokratis,0.0,5,False,48,Papastathopoulos,1,2019-20


## Add GW data

In [12]:
data_merge = []

for iter_season in Setup.GW_folders:
    
    GW_path = Setup.GW_folders[iter_season]

    
    for fname in glob.glob(GW_path):
        
        fname_game_week = fname.replace(GW_path.replace('/*',''),'')
        
        if re.search("^/gw\d+.csv", fname_game_week):

            import_df = pd.read_csv(fname,encoding = "latin1", parse_dates = ['kickoff_time'])

            if import_df.shape[0] > 0:
                
                import_df['FIX_Fixture_date'] = import_df['kickoff_time'].dt.date 
                import_df['file'] = fname
                import_df['FIX_season'] = iter_season
                import_df['FIX_GW'] = fname_game_week.replace('/gw','').replace('.csv','')
                data_merge.append(import_df)
        else:
            print('skip', fname)

player_gws = pd.concat(data_merge, sort = 'True').sort_values(['name','FIX_Fixture_date'])

player_gws = player_gws[['FIX_season',
                         'FIX_Fixture_date',
                         'element',
                         'total_points',
                         'goals_scored',
                         'goals_conceded',
                         'assists',
                         'minutes',
                         'bps',
                         'ict_index',
                         'influence',
                         'creativity',
                         'threat',
                         'clean_sheets',
                         'value']]


player_gws = player_gws.rename(columns = {'opponent_team' : 'FIX_opponent_code',
                                          'element' : 'player_ID',
                                          'value' : 'cost_GW'})
player_gws.tail()

skip /Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2019-20/gws/merged_gw.csv
skip /Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2020-21/gws/xP8.csv
skip /Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2020-21/gws/xP9.csv
skip /Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2020-21/gws/xP10.csv
skip /Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2020-21/gws/xP7.csv
skip /Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2020-21/gws/xP6.csv
skip /Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2020-21/gws/xP4.csv
skip /Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2020-21/gws/xP5.csv
skip /Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2020-21/gws/merged_gw.csv
skip /Users/calumthompson/Documents/Fantasy_football_LOCAL/Github_data/data/2020-21/gws/xP1.csv
skip /Users/calumthompson/D

,FIX_season,FIX_Fixture_date,player_ID,total_points,goals_scored,goals_conceded,assists,minutes,bps,ict_index,influence,creativity,threat,clean_sheets,cost_GW
652,2019-20,2020-07-09,27,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,43
653,2019-20,2020-07-12,27,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,43
660,2019-20,2020-07-16,27,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,43
664,2019-20,2020-07-21,27,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,43
665,2019-20,2020-07-26,27,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,43


## Merge GW and player detail data

In [13]:
player_df = pd.merge(player_gws,player_details, on = ['FIX_season','player_ID'])
player_df.head()

,FIX_season,FIX_Fixture_date,player_ID,total_points,goals_scored,goals_conceded,assists,minutes,bps,ict_index,influence,creativity,threat,clean_sheets,cost_GW,position,first_name,form,in_dreamteam,cost_today,second_name,team_ID
0,2020-21,2020-09-14,78,1,0,2,0,45,-3,3.4,1.2,0.3,32.0,0,55,4,Aaron,1.0,False,53,Connolly,3
1,2020-21,2020-09-20,78,8,1,0,0,89,27,6.9,34.8,11.3,23.0,1,55,4,Aaron,1.0,False,53,Connolly,3
2,2020-21,2020-09-26,78,2,0,2,0,73,2,1.9,0.0,12.1,8.0,0,55,4,Aaron,1.0,False,53,Connolly,3
3,2020-21,2020-10-03,78,2,0,3,0,65,7,0.7,3.0,0.3,4.0,0,55,4,Aaron,1.0,False,53,Connolly,3
4,2020-21,2020-10-18,78,4,0,0,1,12,13,3.0,17.2,10.3,2.0,0,55,4,Aaron,1.0,False,53,Connolly,3


# Merge player details with team details 

In [14]:
dataset_df = pd.merge(fixtures_df, player_df, on = ['FIX_season','team_ID','FIX_Fixture_date'], how = 'left')

# Downfill teams for matches that have yet to take place

In [15]:
# Identify which games need downfilling
future_games = dataset_df.loc[(dataset_df['player_ID'].isna() == True) & (dataset_df['FIX_season'] == Setup.this_season) & (pd.to_datetime(dataset_df['FIX_Fixture_date']) > pd.to_datetime('today'))]
past_games   = dataset_df.loc[(dataset_df['player_ID'].isna() == False)]

# Mark games that have already taken place
past_games.loc[:,'forecast'] = 0 

# Create downfill df
# Start with most recent fixture dates for each team
team_most_recent_fixtures = past_games.groupby('FIX_team')['FIX_Fixture_date'].max()

# Get most recent lineups
merge = []
for team, fixture in team_most_recent_fixtures.iteritems():
    merge.append(past_games.loc[(past_games['FIX_team'] == team) & (past_games['FIX_Fixture_date'] == fixture)])


# Merge and only take columns not provided by fixtures_df
team_lineups = pd.concat(merge).drop(columns = fixtures_df.columns.drop('FIX_team'))

# Create df for future games with team details
downfill_df = pd.merge(future_games[fixtures_df.columns], team_lineups, on = 'FIX_team', how = 'left').sort_values(by = ['FIX_Fixture_date'])

downfill_df.loc[:,'FIX_season'] = Setup.this_season

# Merge with historic data 
dataset_df = pd.concat([past_games,downfill_df],sort=False)
dataset_df = dataset_df.sort_values(['second_name','first_name','FIX_Fixture_date'])

# Mark future games with 1 and 0 for past games
dataset_df['forecast'] = np.where(pd.to_datetime(dataset_df['FIX_Fixture_date']) < pd.to_datetime('today') , 0, 1)

dataset_df.tail()

/Users/calumthompson/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/calumthompson/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,FIX_season,FIX_GW,FIX_Fixture_date,team_ID,FIX_team,FIX_opponent_code,FIX_Opponent,FIX_Home?,difficulty,INCOMING_team_points,INCOMING_opponent_points,INCOMING_team_goals,INCOMING_opponent_goals,INCOMING_points_diff,INCOMING_goals_diff,player_ID,total_points,goals_scored,goals_conceded,assists,minutes,bps,ict_index,influence,creativity,threat,clean_sheets,cost_GW,position,first_name,form,in_dreamteam,cost_today,second_name,forecast
15033,2020-21,34,2021-05-01,9,Leicester,16,Southampton,0,3,19.0,13.0,19.0,11.0,6.0,8.0,541.0,1.0,0.0,0.0,0.0,45.0,8.0,4.6,10.8,25.3,10.0,0.0,59.0,3.0,Cengiz,1.5,False,59.0,Ãnder,1
15062,2020-21,35,2021-05-08,9,Leicester,14,Newcastle,1,2,19.0,13.0,19.0,11.0,6.0,8.0,541.0,1.0,0.0,0.0,0.0,45.0,8.0,4.6,10.8,25.3,10.0,0.0,59.0,3.0,Cengiz,1.5,False,59.0,Ãnder,1
15091,2020-21,36,2021-05-11,9,Leicester,13,Man Utd,0,4,19.0,13.0,19.0,11.0,6.0,8.0,541.0,1.0,0.0,0.0,0.0,45.0,8.0,4.6,10.8,25.3,10.0,0.0,59.0,3.0,Cengiz,1.5,False,59.0,Ãnder,1
15120,2020-21,37,2021-05-15,9,Leicester,5,Chelsea,0,4,19.0,13.0,19.0,11.0,6.0,8.0,541.0,1.0,0.0,0.0,0.0,45.0,8.0,4.6,10.8,25.3,10.0,0.0,59.0,3.0,Cengiz,1.5,False,59.0,Ãnder,1
15149,2020-21,38,2021-05-23,9,Leicester,17,Spurs,1,4,19.0,13.0,19.0,11.0,6.0,8.0,541.0,1.0,0.0,0.0,0.0,45.0,8.0,4.6,10.8,25.3,10.0,0.0,59.0,3.0,Cengiz,1.5,False,59.0,Ãnder,1


# Player recent performance


In [16]:
def generate_performance_col (df, field):
    
    df['LW_' + str(field)] = df.groupby(['second_name','first_name'])[field].shift(periods = 1 )
    
    df['L4W_' + str(field)] = df.groupby(['second_name','first_name'])['LW_' + str(field)].rolling(4, min_periods = 1).mean().reset_index(level=['second_name','first_name'], drop = True)
    df['L8W_' + str(field)] = df.groupby(['second_name','first_name'])['LW_' + str(field)].rolling(8, min_periods = 1).mean().reset_index(level=['second_name','first_name'], drop = True)
    df['EWMA_HL1_' + str(field)] = df.groupby(['second_name','first_name'])['LW_' + str(field)].apply(lambda x: x.ewm(halflife=4).mean())
    df['EWMA_HL2_' + str(field)] = df.groupby(['second_name','first_name'])['LW_' + str(field)].apply(lambda x: x.ewm(halflife=4).mean())
    df['EWMA_HL4_' + str(field)] = df.groupby(['second_name','first_name'])['LW_' + str(field)].apply(lambda x: x.ewm(halflife=4).mean())
    df['EWMA_HL8_' + str(field)] = df.groupby(['second_name','first_name'])['LW_' + str(field)].apply(lambda x: x.ewm(halflife=8).mean())

    # Nulls for future games....
    df['L4W_' + str(field)] = np.where(df['forecast'] > 0, np.nan, df['L4W_' + str(field)])
    df['L8W_' + str(field)] = np.where(df['forecast'] > 0, np.nan, df['L8W_' + str(field)])
    df['EWMA_HL1_' + str(field)] = np.where(df['forecast'] > 0, np.nan, df['EWMA_HL1_' + str(field)])
    df['EWMA_HL2_' + str(field)] = np.where(df['forecast'] > 0, np.nan, df['EWMA_HL2_' + str(field)])
    df['EWMA_HL4_' + str(field)] = np.where(df['forecast'] > 0, np.nan, df['EWMA_HL4_' + str(field)])
    df['EWMA_HL8_' + str(field)] = np.where(df['forecast'] > 0, np.nan, df['EWMA_HL8_' + str(field)])

    # then downfill
    df['L4W_' + str(field)] = df.groupby(['second_name','first_name'])['L4W_' + str(field)].ffill()
    df['L8W_' + str(field)] = df.groupby(['second_name','first_name'])['L8W_' + str(field)].ffill()
    df['EWMA_HL1_' + str(field)]= df.groupby(['second_name','first_name'])['EWMA_HL1_' + str(field)].ffill()
    df['EWMA_HL2_' + str(field)]= df.groupby(['second_name','first_name'])['EWMA_HL2_' + str(field)].ffill()
    df['EWMA_HL4_' + str(field)]= df.groupby(['second_name','first_name'])['EWMA_HL4_' + str(field)].ffill()
    df['EWMA_HL8_' + str(field)]= df.groupby(['second_name','first_name'])['EWMA_HL8_' + str(field)].ffill()
        


In [17]:
performance_cols = ['cost_GW',
                    'total_points',
                    'goals_scored',
                    'minutes',
                    'goals_conceded',
                    'bps',
                    'assists',
                    'ict_index',
                    'influence',
                    'creativity',
                    'threat',
                    'clean_sheets']

for col in performance_cols:
    print("Generating: ", col)
    generate_performance_col(dataset_df, col)
    
dataset_df.head()

Generating:  cost_GW
Generating:  total_points
Generating:  goals_scored
Generating:  minutes
Generating:  goals_conceded
Generating:  bps
Generating:  assists
Generating:  ict_index
Generating:  influence
Generating:  creativity
Generating:  threat
Generating:  clean_sheets


,FIX_season,FIX_GW,FIX_Fixture_date,team_ID,FIX_team,FIX_opponent_code,FIX_Opponent,FIX_Home?,difficulty,INCOMING_team_points,INCOMING_opponent_points,INCOMING_team_goals,INCOMING_opponent_goals,INCOMING_points_diff,INCOMING_goals_diff,player_ID,total_points,goals_scored,goals_conceded,assists,minutes,bps,ict_index,influence,creativity,threat,clean_sheets,cost_GW,position,first_name,form,in_dreamteam,cost_today,second_name,forecast,LW_cost_GW,L4W_cost_GW,L8W_cost_GW,EWMA_HL1_cost_GW,EWMA_HL2_cost_GW,EWMA_HL4_cost_GW,EWMA_HL8_cost_GW,LW_total_points,L4W_total_points,L8W_total_points,EWMA_HL1_total_points,EWMA_HL2_total_points,EWMA_HL4_total_points,EWMA_HL8_total_points,LW_goals_scored,...,EWMA_HL8_goals_conceded,LW_bps,L4W_bps,L8W_bps,EWMA_HL1_bps,EWMA_HL2_bps,EWMA_HL4_bps,EWMA_HL8_bps,LW_assists,L4W_assists,L8W_assists,EWMA_HL1_assists,EWMA_HL2_assists,EWMA_HL4_assists,EWMA_HL8_assists,LW_ict_index,L4W_ict_index,L8W_ict_index,EWMA_HL1_ict_index,EWMA_HL2_ict_index,EWMA_HL4_ict_index,EWMA_HL8_ict_index,LW_influence,L4W_influence,L8W_influence,EWMA_HL1_influence,EWMA_HL2_influence,EWMA_HL4_influence,EWMA_HL8_influence,LW_creativity,L4W_creativity,L8W_creativity,EWMA_HL1_creativity,EWMA_HL2_creativity,EWMA_HL4_creativity,EWMA_HL8_creativity,LW_threat,L4W_threat,L8W_threat,EWMA_HL1_threat,EWMA_HL2_threat,EWMA_HL4_threat,EWMA_HL8_threat,LW_clean_sheets,L4W_clean_sheets,L8W_clean_sheets,EWMA_HL1_clean_sheets,EWMA_HL2_clean_sheets,EWMA_HL4_clean_sheets,EWMA_HL8_clean_sheets
22206,2019-20,1,2019-08-09,14,Norwich,10,Liverpool,0,5,60.0,46.0,44.0,40.0,14.0,4.0,274.0,0.0,0.0,4.0,0.0,90.0,16.0,2.7,25.8,1.4,0.0,0.0,45.0,2.0,Maximillian,0.9,False,42.0,Aarons,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22231,2019-20,2,2019-08-17,14,Norwich,13,Newcastle,1,2,1.0,3.0,1.0,4.0,-2.0,-3.0,274.0,1.0,0.0,1.0,0.0,90.0,7.0,3.6,4.2,14.2,18.0,0.0,45.0,2.0,Maximillian,0.9,False,42.0,Aarons,0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,4.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7,2.70,2.70,2.700000,2.700000,2.700000,2.700000,25.8,25.800000,25.800000,25.800000,25.800000,25.800000,25.800000,1.4,1.4,1.4,1.400000,1.400000,1.400000,1.400000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22256,2019-20,3,2019-08-24,14,Norwich,6,Chelsea,1,4,4.0,1.0,4.0,1.0,3.0,3.0,274.0,1.0,0.0,3.0,0.0,90.0,6.0,2.1,16.6,2.1,2.0,0.0,45.0,2.0,Maximillian,0.9,False,42.0,Aarons,0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,1.0,0.500000,0.500000,0.543214,0.543214,0.543214,0.521647,0.0,...,2.435058,7.0,11.500000,11.500000,11.111077,11.111077,11.111077,11.305174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.6,3.15,3.15,3.188892,3.188892,3.188892,3.169483,4.2,15.000000,15.000000,14.066586,14.066586,14.066586,14.532418,14.2,7.8,7.8,8.353134,8.353134,8.353134,8.077086,18.0,9.000000,9.000000,9.777845,9.777845,9.777845,9.389652,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22281,2019-20,4,2019-08-31,14,Norwich,19,West Ham,0,2,6.0,4.0,6.0,4.0,2.0,2.0,274.0,0.0,0.0,2.0,0.0,90.0,11.0,4.4,11.8,12.3,20.0,0.0,45.0,2.0,Maximillian,0.9,False,42.0,Aarons,0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,1.0,0.666667,0.666667,0.722486,0.722486,0.722486,0.695095,0.0,...,2.639903,6.0,9.666667,9.666667,9.105163,9.105163,9.105163,9.381546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1,2.80,2.80,2.761541,2.761541,2.761541,2.781694,16.6,15.533333,15.533333,15.060860,15.060860,15.060860,15.282112,2.1,5.9,5.9,5.899003,5.899003,5.899003,5.909826,2.0,6.666667,6.666667,6.725319,6.725319,6.725319,6.710203,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22307,2019-20,5,2019-09-14,14,Norwich,11,Man City,1,4,6.0,7.0,6.0,6.0,-1.0,0.0,274.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,2.0,Maximillian,0.9,False,42.0,Aarons,0,45.0,45.0,

# Tidy

In [18]:
dataset_df = dataset_df[['FIX_season',
                         'FIX_GW',
                         'FIX_Fixture_date',                         
                         'player_ID',
                         'first_name',
                         'second_name',
                         'forecast',
                         'position',
                         'team_ID',
                         'FIX_team',
                         'FIX_opponent_code',
                         'FIX_Opponent',
                         'FIX_Home?',
                         'difficulty',
                         'total_points',
                         'cost_today',
                         'cost_GW',
                         'INCOMING_team_points',
                         'INCOMING_opponent_points',
                         'INCOMING_team_goals',
                         'INCOMING_opponent_goals',
                         'INCOMING_points_diff',
                         'INCOMING_goals_diff',
#                          'minutes',
#                          'goals_scored',
#                          'goals_conceded',
#                          'assists',
#                          'bps',
#                          'ict_index',
#                          'influence',
#                          'creativity',
#                          'threat',
#                          'clean_sheets',
                         'form',
                         'in_dreamteam',                         
                         'LW_cost_GW',
                         'L4W_cost_GW',
                         'L8W_cost_GW',
                         'EWMA_HL1_cost_GW',
                         'EWMA_HL2_cost_GW',
                         'EWMA_HL4_cost_GW',
                         'EWMA_HL8_cost_GW',
                         'LW_total_points',
                         'L4W_total_points',
                         'L8W_total_points',
                         'EWMA_HL1_total_points',
                         'EWMA_HL2_total_points',
                         'EWMA_HL4_total_points',
                         'EWMA_HL8_total_points',
                         'LW_goals_scored',
                         'L4W_goals_scored',
                         'L8W_goals_scored',
                         'EWMA_HL1_goals_scored',
                         'EWMA_HL2_goals_scored',
                         'EWMA_HL4_goals_scored',
                         'EWMA_HL8_goals_scored',
                         'LW_minutes',
                         'L4W_minutes',
                         'L8W_minutes',
                         'EWMA_HL1_minutes',
                         'EWMA_HL2_minutes',
                         'EWMA_HL4_minutes',
                         'EWMA_HL8_minutes',
                         'LW_goals_conceded',
                         'L4W_goals_conceded',
                         'L8W_goals_conceded',
                         'EWMA_HL1_goals_conceded',
                         'EWMA_HL2_goals_conceded',
                         'EWMA_HL4_goals_conceded',
                         'EWMA_HL8_goals_conceded',
                         'LW_bps',
                         'L4W_bps',
                         'L8W_bps',
                         'EWMA_HL1_bps',
                         'EWMA_HL2_bps',
                         'EWMA_HL4_bps',
                         'EWMA_HL8_bps',
                         'LW_assists',
                         'L4W_assists',
                         'L8W_assists',
                         'EWMA_HL1_assists',
                         'EWMA_HL2_assists',
                         'EWMA_HL4_assists',
                         'EWMA_HL8_assists',
                         'LW_ict_index',
                         'L4W_ict_index',
                         'L8W_ict_index',
                         'EWMA_HL1_ict_index',
                         'EWMA_HL2_ict_index',
                         'EWMA_HL4_ict_index',
                         'EWMA_HL8_ict_index',
                         'LW_influence',
                         'L4W_influence',
                         'L8W_influence',
                         'EWMA_HL1_influence',
                         'EWMA_HL2_influence',
                         'EWMA_HL4_influence',
                         'EWMA_HL8_influence',
                         'LW_creativity',
                         'L4W_creativity',
                         'L8W_creativity',
                         'EWMA_HL1_creativity',
                         'EWMA_HL2_creativity',
                         'EWMA_HL4_creativity',
                         'EWMA_HL8_creativity',
                         'LW_threat',
                         'L4W_threat',
                         'L8W_threat',
                         'EWMA_HL1_threat',
                         'EWMA_HL2_threat',
                         'EWMA_HL4_threat',
                         'EWMA_HL8_threat',
                         'LW_clean_sheets',
                         'L4W_clean_sheets',
                         'L8W_clean_sheets',
                         'EWMA_HL1_clean_sheets',
                         'EWMA_HL2_clean_sheets',
                         'EWMA_HL4_clean_sheets',
                         'EWMA_HL8_clean_sheets']].sort_values(by = ['team_ID','player_ID','FIX_Fixture_date'])

dataset_df.tail()

,FIX_season,FIX_GW,FIX_Fixture_date,player_ID,first_name,second_name,forecast,position,team_ID,FIX_team,FIX_opponent_code,FIX_Opponent,FIX_Home?,difficulty,total_points,cost_today,cost_GW,INCOMING_team_points,INCOMING_opponent_points,INCOMING_team_goals,INCOMING_opponent_goals,INCOMING_points_diff,INCOMING_goals_diff,form,in_dreamteam,LW_cost_GW,L4W_cost_GW,L8W_cost_GW,EWMA_HL1_cost_GW,EWMA_HL2_cost_GW,EWMA_HL4_cost_GW,EWMA_HL8_cost_GW,LW_total_points,L4W_total_points,L8W_total_points,EWMA_HL1_total_points,EWMA_HL2_total_points,EWMA_HL4_total_points,EWMA_HL8_total_points,LW_goals_scored,L4W_goals_scored,L8W_goals_scored,EWMA_HL1_goals_scored,EWMA_HL2_goals_scored,EWMA_HL4_goals_scored,EWMA_HL8_goals_scored,LW_minutes,L4W_minutes,L8W_minutes,EWMA_HL1_minutes,...,EWMA_HL8_goals_conceded,LW_bps,L4W_bps,L8W_bps,EWMA_HL1_bps,EWMA_HL2_bps,EWMA_HL4_bps,EWMA_HL8_bps,LW_assists,L4W_assists,L8W_assists,EWMA_HL1_assists,EWMA_HL2_assists,EWMA_HL4_assists,EWMA_HL8_assists,LW_ict_index,L4W_ict_index,L8W_ict_index,EWMA_HL1_ict_index,EWMA_HL2_ict_index,EWMA_HL4_ict_index,EWMA_HL8_ict_index,LW_influence,L4W_influence,L8W_influence,EWMA_HL1_influence,EWMA_HL2_influence,EWMA_HL4_influence,EWMA_HL8_influence,LW_creativity,L4W_creativity,L8W_creativity,EWMA_HL1_creativity,EWMA_HL2_creativity,EWMA_HL4_creativity,EWMA_HL8_creativity,LW_threat,L4W_threat,L8W_threat,EWMA_HL1_threat,EWMA_HL2_threat,EWMA_HL4_threat,EWMA_HL8_threat,LW_clean_sheets,L4W_clean_sheets,L8W_clean_sheets,EWMA_HL1_clean_sheets,EWMA_HL2_clean_sheets,EWMA_HL4_clean_sheets,EWMA_HL8_clean_sheets
28501,2019-20,34,2020-07-08,652.0,Leonardo,Campana,0,4.0,20,Wolves,15,Sheffield Utd,0,3,0.0,45.0,45.0,59.0,53.0,45.0,49.0,6.0,-4.0,0.0,False,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28532,2019-20,35,2020-07-12,652.0,Leonardo,Campana,0,4.0,20,Wolves,8,Everton,1,2,0.0,45.0,45.0,59.0,52.0,45.0,35.0,7.0,10.0,0.0,False,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28563,2019-20,36,2020-07-15,652.0,Leonardo,Campana,0,4.0,20,Wolves,5,Burnley,0,3,0.0,45.0,45.0,62.0,55.0,48.0,41.0,7.0,7.0,0.0,False,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28594,2019-20,37,2020-07-20,652.0,Leonardo,Campana,0,4.0,20,Wolves,7,Crystal Palace,1,3,0.0,45.0,45.0,63.0,57.0,49.0,40.0,6.0,9.0,0.0,False,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28625,2019-20,38,2020-07-26,652.0,Leonardo,Campana,0,4.0,20,Wolves,6,Chelsea,0,4,0.0,45.0,45.0,66.0,46.0,51.0,30.0,20.0,21.0,0.0,False,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Export

In [19]:
dataset_df['FIX_season'].unique()

array(['2019-20', '2020-21'], dtype=object)

In [20]:
dataset_df.to_csv(Setup.input_data_folder + str(date.today()) + '.csv', index = False)

# Sense check

In [21]:
dataset_df.loc[dataset_df['second_name'] == 'Son']

,FIX_season,FIX_GW,FIX_Fixture_date,player_ID,first_name,second_name,forecast,position,team_ID,FIX_team,FIX_opponent_code,FIX_Opponent,FIX_Home?,difficulty,total_points,cost_today,cost_GW,INCOMING_team_points,INCOMING_opponent_points,INCOMING_team_goals,INCOMING_opponent_goals,INCOMING_points_diff,INCOMING_goals_diff,form,in_dreamteam,LW_cost_GW,L4W_cost_GW,L8W_cost_GW,EWMA_HL1_cost_GW,EWMA_HL2_cost_GW,EWMA_HL4_cost_GW,EWMA_HL8_cost_GW,LW_total_points,L4W_total_points,L8W_total_points,EWMA_HL1_total_points,EWMA_HL2_total_points,EWMA_HL4_total_points,EWMA_HL8_total_points,LW_goals_scored,L4W_goals_scored,L8W_goals_scored,EWMA_HL1_goals_scored,EWMA_HL2_goals_scored,EWMA_HL4_goals_scored,EWMA_HL8_goals_scored,LW_minutes,L4W_minutes,L8W_minutes,EWMA_HL1_minutes,...,EWMA_HL8_goals_conceded,LW_bps,L4W_bps,L8W_bps,EWMA_HL1_bps,EWMA_HL2_bps,EWMA_HL4_bps,EWMA_HL8_bps,LW_assists,L4W_assists,L8W_assists,EWMA_HL1_assists,EWMA_HL2_assists,EWMA_HL4_assists,EWMA_HL8_assists,LW_ict_index,L4W_ict_index,L8W_ict_index,EWMA_HL1_ict_index,EWMA_HL2_ict_index,EWMA_HL4_ict_index,EWMA_HL8_ict_index,LW_influence,L4W_influence,L8W_influence,EWMA_HL1_influence,EWMA_HL2_influence,EWMA_HL4_influence,EWMA_HL8_influence,LW_creativity,L4W_creativity,L8W_creativity,EWMA_HL1_creativity,EWMA_HL2_creativity,EWMA_HL4_creativity,EWMA_HL8_creativity,LW_threat,L4W_threat,L8W_threat,EWMA_HL1_threat,EWMA_HL2_threat,EWMA_HL4_threat,EWMA_HL8_threat,LW_clean_sheets,L4W_clean_sheets,L8W_clean_sheets,EWMA_HL1_clean_sheets,EWMA_HL2_clean_sheets,EWMA_HL4_clean_sheets,EWMA_HL8_clean_sheets
12857,2019-20,1,2019-08-10,342.0,Heung-Min,Son,0,3.0,17,Spurs,2,Aston Villa,1,2,0.0,97.0,95.0,44.0,61.0,36.0,56.0,-17.0,-20.0,5.6,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12884,2019-20,2,2019-08-17,342.0,Heung-Min,Son,0,3.0,17,Spurs,11,Man City,0,4,0.0,97.0,95.0,3.0,1.0,3.0,1.0,2.0,2.0,5.6,False,95.0,95.00,95.000000,95.000000,95.000000,95.000000,95.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
12911,2019-20,3,2019-08-25,342.0,Heung-Min,Son,0,3.0,17,Spurs,13,Newcastle,1,2,2.0,97.0,95.0,4.0,4.0,5.0,7.0,0.0,-2.0,5.6,False,95.0,95.00,95.000000,95.000000,95.000000,95.000000,95.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
12938,2019-20,4,2019-09-01,342.0,Heung-Min,Son,0,3.0,17,Spurs,1,Arsenal,0,4,5.0,97.0,95.0,4.0,4.0,5.0,2.0,0.0,3.0,5.6,False,95.0,95.00,95.000000,95.000000,95.000000,95.000000,95.000000,2.0,0.666667,0.666667,0.784928,0.784928,0.784928,0.725189,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,90.0,30.00,30.000000,35.321777,...,0.362595,9.0,3.00,3.000000,3.532178,3.532178,3.532178,3.263352,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,12.4,4.133333,4.133333,4.866556,4.866556,4.866556,4.496174,13.4,4.466667,4.466667,5.2

In [22]:
dataset_df.loc[dataset_df['second_name'] == 'Kane']

,FIX_season,FIX_GW,FIX_Fixture_date,player_ID,first_name,second_name,forecast,position,team_ID,FIX_team,FIX_opponent_code,FIX_Opponent,FIX_Home?,difficulty,total_points,cost_today,cost_GW,INCOMING_team_points,INCOMING_opponent_points,INCOMING_team_goals,INCOMING_opponent_goals,INCOMING_points_diff,INCOMING_goals_diff,form,in_dreamteam,LW_cost_GW,L4W_cost_GW,L8W_cost_GW,EWMA_HL1_cost_GW,EWMA_HL2_cost_GW,EWMA_HL4_cost_GW,EWMA_HL8_cost_GW,LW_total_points,L4W_total_points,L8W_total_points,EWMA_HL1_total_points,EWMA_HL2_total_points,EWMA_HL4_total_points,EWMA_HL8_total_points,LW_goals_scored,L4W_goals_scored,L8W_goals_scored,EWMA_HL1_goals_scored,EWMA_HL2_goals_scored,EWMA_HL4_goals_scored,EWMA_HL8_goals_scored,LW_minutes,L4W_minutes,L8W_minutes,EWMA_HL1_minutes,...,EWMA_HL8_goals_conceded,LW_bps,L4W_bps,L8W_bps,EWMA_HL1_bps,EWMA_HL2_bps,EWMA_HL4_bps,EWMA_HL8_bps,LW_assists,L4W_assists,L8W_assists,EWMA_HL1_assists,EWMA_HL2_assists,EWMA_HL4_assists,EWMA_HL8_assists,LW_ict_index,L4W_ict_index,L8W_ict_index,EWMA_HL1_ict_index,EWMA_HL2_ict_index,EWMA_HL4_ict_index,EWMA_HL8_ict_index,LW_influence,L4W_influence,L8W_influence,EWMA_HL1_influence,EWMA_HL2_influence,EWMA_HL4_influence,EWMA_HL8_influence,LW_creativity,L4W_creativity,L8W_creativity,EWMA_HL1_creativity,EWMA_HL2_creativity,EWMA_HL4_creativity,EWMA_HL8_creativity,LW_threat,L4W_threat,L8W_threat,EWMA_HL1_threat,EWMA_HL2_threat,EWMA_HL4_threat,EWMA_HL8_threat,LW_clean_sheets,L4W_clean_sheets,L8W_clean_sheets,EWMA_HL1_clean_sheets,EWMA_HL2_clean_sheets,EWMA_HL4_clean_sheets,EWMA_HL8_clean_sheets
12855,2019-20,1,2019-08-10,338.0,Harry,Kane,0,4.0,17,Spurs,2,Aston Villa,1,2,13.0,110.0,110.0,44.0,61.0,36.0,56.0,-17.0,-20.0,6.7,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12882,2019-20,2,2019-08-17,338.0,Harry,Kane,0,4.0,17,Spurs,11,Man City,0,4,2.0,110.0,110.0,3.0,1.0,3.0,1.0,2.0,2.0,6.7,False,110.0,110.00,110.000000,110.000000,110.000000,110.000000,110.000000,13.0,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,2.0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,90.0,90.00,90.000000,90.000000,...,1.000000,50.0,50.00,50.000000,50.000000,50.000000,50.000000,50.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,18.3,1.830000e+01,18.300000,18.300000,18.300000,18.300000,18.300000,64.2,6.420000e+01,64.200000,64.200000,64.200000,64.200000,64.200000,24.1,24.100000,24.100000,24.100000,24.100000,24.100000,24.100000,95.0,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
12909,2019-20,3,2019-08-25,338.0,Harry,Kane,0,4.0,17,Spurs,13,Newcastle,1,2,2.0,110.0,110.0,4.0,4.0,5.0,7.0,0.0,-2.0,6.7,False,110.0,110.00,110.000000,110.000000,110.000000,110.000000,110.000000,2.0,7.500000,7.500000,7.024650,7.024650,7.024650,7.261880,0.0,1.000000,1.000000,0.913573,0.913573,0.913573,0.956705,90.0,90.00,90.000000,90.000000,...,1.521647,4.0,27.00,27.000000,25.012174,25.012174,25.012174,26.004224,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.6,9.450000e+00,9.450000,8.685119,8.685119,8.685119,9.066843,0.4,3.230000e+01,32.300000,29.542971,29.542971,29.542971,30.918902,1.8,12.950000,12.950000,11.986336,11.986336,11.986336,12.467265,4.0,49.500000,49.500000,45.567561,45.567561,45.567561,47.530095,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
12936,2019-20,4,2019-09-01,338.0,Harry,Kane,0,4.0,17,Spurs,1,Arsenal,0,4,6.0,110.0,111.0,4.0,4.0,5.0,2.0,0.0,3.0,6.7,False,110.0,110.00,110.000000,110.000000,110.000000,110.000000,110.000000,2.0,5.666667,5.666667,5.052655,5.052655,5.052655,5.353950,0.0,0.666667,0.666667,0.555028,0.555028,0.555028,0.609809,90.0,90.00,90.000000,90.000000,...,1.332501,0.0,18.00,18.000000,15.195791,15.195791,15.195791,16.575231,0.0,0.00